In [1]:
import pandas as pd
import numpy as np
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
!ls

README.md                 scac.csv
all_stocks_5yr.csv        stock-fraud-predict.ipynb


In [3]:
df = pd.read_csv('scac.csv')

In [4]:
df.head()

,filing_name,filing_date,district_court,exchange,ticker
0,"ILG, Inc.",07/06/2018,D. Delaware,NASDAQ,ILG
1,KLX Inc.,07/06/2018,D. Delaware,NASDAQ,KLXI
2,Triangle Capital Corporation,07/06/2018,E.D. North Carolina,New York SE,TCAP
3,Vectren Corporation,07/05/2018,S.D. Indiana,New York SE,VVC
4,DCT Industrial Trust Inc.,07/02/2018,D. Colorado,New York SE,DCT


In [5]:
df1 = pd.read_csv('all_stocks_5yr.csv')

In [6]:
df1.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619040 entries, 0 to 619039
Data columns (total 7 columns):
date      619040 non-null object
open      619029 non-null float64
high      619032 non-null float64
low       619032 non-null float64
close     619040 non-null float64
volume    619040 non-null int64
Name      619040 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 33.1+ MB


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 5 columns):
filing_name       4996 non-null object
filing_date       4996 non-null object
district_court    4996 non-null object
exchange          4996 non-null object
ticker            4996 non-null object
dtypes: object(5)
memory usage: 195.2+ KB


In [16]:
df1.rename(columns={'Name':'ticker'}, inplace=True)

In [17]:
df1.head()

,date,open,high,low,close,volume,ticker
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [35]:
df_merge = pd.merge(df, df1, on='ticker')

In [36]:
df.head()

,filing_name,filing_date,district_court,exchange,ticker
0,"ILG, Inc.",07/06/2018,D. Delaware,NASDAQ,ILG
1,KLX Inc.,07/06/2018,D. Delaware,NASDAQ,KLXI
2,Triangle Capital Corporation,07/06/2018,E.D. North Carolina,New York SE,TCAP
3,Vectren Corporation,07/05/2018,S.D. Indiana,New York SE,VVC
4,DCT Industrial Trust Inc.,07/02/2018,D. Colorado,New York SE,DCT


In [42]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619040 entries, 0 to 619039
Data columns (total 7 columns):
date      619040 non-null object
open      619029 non-null float64
high      619032 non-null float64
low       619032 non-null float64
close     619040 non-null float64
volume    619040 non-null int64
ticker    619040 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 33.1+ MB


In [38]:
df_merge.head()

,filing_name,filing_date,district_court,exchange,ticker,date,open,high,low,close,volume
0,Express Scripts Holding Company,06/22/2018,E.D. Missouri,NASDAQ,ESRX,2013-02-08,55.27,55.92,55.12,55.74,3413611
1,Express Scripts Holding Company,06/22/2018,E.D. Missouri,NASDAQ,ESRX,2013-02-11,56.04,56.04,54.88,55.05,3512527
2,Express Scripts Holding Company,06/22/2018,E.D. Missouri,NASDAQ,ESRX,2013-02-12,55.01,55.29,54.24,54.34,5723673
3,Express Scripts Holding Company,06/22/2018,E.D. Missouri,NASDAQ,ESRX,2013-02-13,54.35,55.47,54.35,55.43,4581808
4,Express Scripts Holding Company,06/22/2018,E.D. Missouri,NASDAQ,ESRX,2013-02-14,55.26,55.88,55.10,55.10,7916922


In [41]:
df_merge.exchange.value_counts()

New York SE         378430
NASDAQ              135562
American SE           2518
ETF                   1259
Privately Traded      1259
Name: exchange, dtype: int64

In [40]:
df_merge.filing_name.value_counts()

Juniper Networks, Inc.                                                         6295
General Electric Company                                                       5036
Chesapeake Energy Corporation                                                  5036
Medtronic, Inc.                                                                5036
Advanced Micro Devices, Inc.                                                   3777
Merck & Co., Inc.                                                              3777
Ford Motor Company                                                             3777
Chipotle Mexican Grill, Inc.                                                   3777
American Express Company                                                       3777
Facebook, Inc.                                                                 3777
General Motors Company                                                         3777
The Boeing Company                                                          

In [43]:
from datetime import datetime

In [46]:
df.filing_date = pd.to_datetime(df['filing_date'])

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 5 columns):
filing_name       4996 non-null object
filing_date       4996 non-null datetime64[ns]
district_court    4996 non-null object
exchange          4996 non-null object
ticker            4996 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 195.2+ KB


In [48]:
df1.date = pd.to_datetime(df1['date'])

In [49]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619040 entries, 0 to 619039
Data columns (total 7 columns):
date      619040 non-null datetime64[ns]
open      619029 non-null float64
high      619032 non-null float64
low       619032 non-null float64
close     619040 non-null float64
volume    619040 non-null int64
ticker    619040 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 33.1+ MB


In [50]:
df_merge = pd.merge(df, df1, on='ticker')

In [51]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 519028 entries, 0 to 519027
Data columns (total 11 columns):
filing_name       519028 non-null object
filing_date       519028 non-null datetime64[ns]
district_court    519028 non-null object
exchange          519028 non-null object
ticker            519028 non-null object
date              519028 non-null datetime64[ns]
open              519022 non-null float64
high              519023 non-null float64
low               519023 non-null float64
close             519028 non-null float64
volume            519028 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(1), object(4)
memory usage: 47.5+ MB
